In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

import time, random, re
import pandas as pd

In [35]:
driver = webdriver.Chrome()

In [36]:
sequence = []
page_num = 1

In [37]:
# 사진 등록된 차량들에서 url에 들어가는 sequence 값 추출
while True:
    url = f'https://www.kbchachacha.com/public/search/main.kbc#!?countryOrder=2&page={page_num}&sort=-orderDate&useCode=002006'
    driver.get(url)

    time.sleep(random.uniform(2, 5))
    items = driver.find_elements(By.CSS_SELECTOR, '.area')

    # 사진 등록이 안된 차량 매물이 나오는 시점에서 break
    if not items:
        break

    # 같은 클래스 이름을 공유하지만 data-car-seq 정보가 없다면 추가 X
    for item in items:
        seq = item.get_attribute('data-car-seq')
        if seq is not None:
            sequence.append(seq)

    page_num += 1

In [38]:
# sequence 값 csv 형식으로 저장
seq_df = pd.DataFrame(sequence, columns=['시퀀스']).reset_index(drop=True)
seq_df.to_csv('seq_data.csv', encoding='utf-8-sig', index_label='번호')

In [39]:
data = []

In [40]:
# sequence 값으로 각 차량들의 페이지에 접속하여 필요한 정보들을 크롤링
for seq in sequence:
    temp = []
    try:
        url = f'https://www.kbchachacha.com/public/car/detail.kbc?carSeq={seq}'
        driver.get(url)

        time.sleep(random.uniform(2, 5))

        place = driver.find_element(By.CSS_SELECTOR, '.txt-info > span:nth-of-type(4)').text

        name = driver.find_element(By.CSS_SELECTOR, '.car-buy-name').text.replace('\n', ' ')
        name = re.sub(r"^\([^)]*\)", "", name).strip()

        price = driver.find_element(By.CSS_SELECTOR, '.car-buy-price > div > dl > dd > strong').text

        relief = driver.find_element(By.CSS_SELECTOR, '.price-tooltip-wrap > strong > strong').text

        info_list = driver.find_elements(By.CSS_SELECTOR, '.detail-info01 > table > tbody > tr > td')
        info = [data.text for data in info_list]
        info = info[:5] + [info[8]]

        car_image = driver.find_element(By.CSS_SELECTOR, '#bx-pager > .page01 > a:nth-child(1) > img')
        image_url = car_image.get_attribute('src')

        # 시퀀스, 모델명, 지역, 가격, 시세안심지수, 번호판, 연식, 주행거리, 연료, 변속기, 색상, 사진
        temp.append(seq)
        temp.append(name)
        temp.append(place)
        temp.append(price)
        temp.append(relief)
        temp.extend(info)
        temp.append(image_url)

        # data 리스트에 car_df에 들어갈 행들을 추가
        data.append(temp)

    # TimeoutException 처리
    except TimeoutException:
        print(f"TimeoutException 발생 - {seq} 페이지 로드 실패")
        time.sleep(10)  # 10초 대기 후 재시도

    # StaleElementReferenceException 처리
    except StaleElementReferenceException:
        print(f"StaleElementReferenceException 발생 - {seq} 페이지 요소 오류")
        time.sleep(5)  # 5초 대기 후 재시도

    # Exception 처리
    except Exception as e:
        print(f"알 수 없는 오류 발생 - {seq} : {e}")

driver.quit()

알 수 없는 오류 발생 - 26626412 : Message: no such element: Unable to locate element: {"method":"css selector","selector":".car-buy-price > div > dl > dd > strong"}
  (Session info: chrome=132.0.6834.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF700AA02F5+28725]
	(No symbol) [0x00007FF700A02AE0]
	(No symbol) [0x00007FF70089510A]
	(No symbol) [0x00007FF7008E93D2]
	(No symbol) [0x00007FF7008E95FC]
	(No symbol) [0x00007FF700933407]
	(No symbol) [0x00007FF70090FFEF]
	(No symbol) [0x00007FF700930181]
	(No symbol) [0x00007FF70090FD53]
	(No symbol) [0x00007FF7008DA0E3]
	(No symbol) [0x00007FF7008DB471]
	GetHandleVerifier [0x00007FF700DCF30D+3366989]
	GetHandleVerifier [0x00007FF700DE12F0+3440688]
	GetHandleVerifier [0x00007FF700DD78FD+3401277]
	GetHandleVerifier [0x00007FF700B6AAAB+858091]
	(No symbol) [0x00007FF700A0E74F]
	(No symbol) [0x00007FF700A0A304]


In [41]:
# data를 csv 형식으로 저장
car_df = pd.DataFrame(data, columns=['시퀀스', '모델명', '지역', '가격', '시세안심지수', '번호판', '연식', '주행거리', '연료', '변속기', '색상', '사진']).reset_index(drop=True)
car_df.to_csv('car_data.csv', encoding='utf-8-sig')